In [ ]:
# ================================
# CNN-LSTM 회귀 학습 전용 스크립트
# - 입력  C:\ESG_Project1\file\solar_data_file\남동발전량_지역매핑_시간행_기상병합.csv
# - 타깃  "발전량(MWh)" 기본값
# - 출력  같은 폴더에 cnn_lstm_solar.pt 저장
# - 기능  범주형 원핫 숫자 표준화 시계열 윈도우 시간순 분할 조기 종료
# ================================
import os
import math
import time
import numpy as np
import pandas as pd
from typing import Tuple, List

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

# ===== 사용자 설정 =====
CSV_PATH = r"C:\ESG_Project1\file\solar_data_file\남동발전량_지역매핑_시간행_기상병합.csv"
TARGET   = "발전량(MWh)"       # 타깃 열 이름
SEQ_LEN  = 24                  # 입력 시퀀스 길이
HORIZON  = 1                   # 예측 지평 1시간
BATCH    = 256
EPOCHS   = 200
LR       = 1e-3
VAL_RATIO = 0.1                # 마지막 10%를 검증으로 사용
PATIENCE = 20                  # 조기 종료 인내
NUM_WORKERS = 0                # Windows 환경 안전값

USE_GPU  = torch.cuda.is_available()
AMP      = True                # 혼합정밀 학습 사용
MODEL_DIR = os.path.dirname(CSV_PATH)
MODEL_PATH = os.path.join(MODEL_DIR, "cnn_lstm_solar.pt")

# ===== 데이터 적재 및 전처리 =====
df = pd.read_csv(CSV_PATH)

# Datetime 열 자동 탐지
dt_col = None
cands = [c for c in df.columns if any(k in str(c).lower() for k in ["datetime","일시","시간","timestamp","ts","측정시각"])]
if cands:
    dt_col = cands[0]
    try:
        df[dt_col] = pd.to_datetime(df[dt_col])
        df = df.sort_values(dt_col).reset_index(drop=True)
    except Exception:
        dt_col = None

if dt_col is None:
    # 일자 + 시 조합
    date_like = [c for c in df.columns if any(k in str(c).lower() for k in ["일자","날짜","date"])]
    hour_like = [c for c in df.columns if any(k in str(c).lower() for k in ["시","hour","시간"])]
    if date_like and hour_like:
        dcol, hcol = date_like[0], hour_like[0]
        df["__dt__"] = pd.to_datetime(df[dcol]) + pd.to_timedelta(df[hcol].astype(int), unit="h")
        dt_col = "__dt__"
        df = df.sort_values(dt_col).reset_index(drop=True)

# 시간 파생 특성
time_feats = []
if dt_col is not None:
    df["hour"]      = df[dt_col].dt.hour
    df["dayofweek"] = df[dt_col].dt.dayofweek
    df["month"]     = df[dt_col].dt.month
    df["is_weekend"]= (df["dayofweek"] >= 5).astype(int)
    time_feats = ["hour","dayofweek","month","is_weekend"]

# 타깃 확인
if TARGET not in df.columns:
    num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
    assert len(num_cols_all) > 0, "숫자형 타깃 후보가 없습니다. TARGET 이름을 확인하세요."
    TARGET = num_cols_all[-1]

# 학습에 불필요한 열 제거
drop_cols = []
for c in df.columns:
    if "Unnamed" in str(c) or "index" == str(c).lower() or "id" == str(c).lower():
        drop_cols.append(c)
if dt_col is not None:
    drop_cols.append(dt_col)

# 범주형 원핫
cat_cols = df.drop(columns=[TARGET] + drop_cols, errors="ignore").select_dtypes(exclude=[np.number]).columns.tolist()
df_cat = pd.get_dummies(df[cat_cols].fillna("MISSING"), dummy_na=False) if cat_cols else pd.DataFrame(index=df.index)

# 수치 컬럼
num_cols = df.drop(columns=[TARGET] + drop_cols + cat_cols, errors="ignore").select_dtypes(include=[np.number]).columns.tolist()
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

# 최종 특성 행렬
X_df = pd.concat([df[num_cols], df_cat, df[time_feats]] if time_feats else [df[num_cols], df_cat], axis=1)
y_df = df[TARGET].astype(float)

# 시간순 분할 인덱스
n = len(X_df)
assert n > SEQ_LEN + HORIZON, "시퀀스 길이보다 데이터가 더 필요합니다."
split_idx = int(n * (1 - VAL_RATIO))
X_train_df, X_val_df = X_df.iloc[:split_idx].copy(), X_df.iloc[split_idx:].copy()
y_train,   y_val     = y_df.iloc[:split_idx].copy(), y_df.iloc[split_idx:].copy()

# 스케일러는 훈련 구간에만 적합
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_df.values)
X_val   = scaler.transform(X_val_df.values)

feature_dim = X_train.shape[1]

# ===== PyTorch 데이터셋 =====
class SeqDataset(Dataset):
    def __init__(self, X: np.ndarray, y: np.ndarray, seq_len: int, horizon: int):
        self.X = X
        self.y = y
        self.seq_len = seq_len
        self.h = horizon
        self.max_i = len(X) - seq_len - horizon + 1
        self.max_i = max(self.max_i, 0)
    def __len__(self):
        return self.max_i
    def __getitem__(self, idx: int):
        x_seq = self.X[idx: idx + self.seq_len]                  # [L, F]
        y_tgt = self.y[idx + self.seq_len + self.h - 1]          # 스칼라
        return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_tgt, dtype=torch.float32)

train_ds = SeqDataset(X_train, y_train.values, SEQ_LEN, HORIZON)
val_ds   = SeqDataset(X_val,   y_val.values,   SEQ_LEN, HORIZON)
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=USE_GPU)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=USE_GPU)

# ===== 모델 정의 =====
class CNNLSTM(nn.Module):
    def __init__(self, in_features: int, seq_len: int,
                 conv_channels: int = 64, lstm_hidden: int = 128, lstm_layers: int = 2, dropout: float = 0.2):
        super().__init__()
        # 입력 [B, L, F] → Conv1d는 [B, C, L] 형식이므로 채널을 F로 보고 시퀀스 축을 길이로 사용
        self.bn_in = nn.BatchNorm1d(in_features)
        self.conv = nn.Sequential(
            nn.Conv1d(in_features, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(conv_channels, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(output_size=seq_len // 2)
        )
        self.lstm = nn.LSTM(input_size=conv_channels, hidden_size=lstm_hidden,
                            num_layers=lstm_layers, batch_first=True, dropout=dropout)
        self.head = nn.Sequential(
            nn.Linear(lstm_hidden, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, 1)
        )
    def forward(self, x):                       # x [B, L, F]
        x = x.transpose(1, 2)                  # [B, F, L]
        x = self.bn_in(x)                      # [B, F, L]
        x = self.conv(x)                       # [B, C, L']
        x = x.transpose(1, 2)                  # [B, L', C]
        out, _ = self.lstm(x)                  # [B, L', H]
        out = out[:, -1, :]                    # [B, H]
        y = self.head(out).squeeze(-1)         # [B]
        return y

device = torch.device("cuda" if USE_GPU else "cpu")
model = CNNLSTM(in_features=feature_dim, seq_len=SEQ_LEN).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
criterion = nn.L1Loss()  # MAE 기반 학습. 필요 시 nn.MSELoss 로 교체
scaler_t = torch.cuda.amp.GradScaler(enabled=AMP and USE_GPU)

# ===== 학습 루프  평가 출력 없음  조기 종료만 =====
best_val = math.inf
wait = 0

for epoch in range(1, EPOCHS + 1):
    model.train()
    for xb, yb in train_loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=AMP and USE_GPU):
            pred = model(xb)
            loss = criterion(pred, yb)
        scaler_t.scale(loss).backward()
        scaler_t.step(optimizer)
        scaler_t.update()
    scheduler.step()

    # 검증 손실만 계산  출력은 하지 않음
    model.eval()
    val_loss = 0.0
    n_val = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=AMP and USE_GPU):
                pred = model(xb)
                l = criterion(pred, yb).item()
            val_loss += l * xb.size(0)
            n_val += xb.size(0)
    val_loss /= max(n_val, 1)

    # 조기 종료
    if val_loss + 1e-8 < best_val:
        best_val = val_loss
        wait = 0
        torch.save({
            "model_state": model.state_dict(),
            "scaler_mean": scaler.mean_.astype(np.float32),
            "scaler_scale": scaler.scale_.astype(np.float32),
            "feature_columns": X_df.columns.tolist(),
            "seq_len": SEQ_LEN,
            "horizon": HORIZON,
            "in_features": feature_dim,
        }, MODEL_PATH)
    else:
        wait += 1
        if wait >= PATIENCE:
            break

print(f"✅ 모델 저장 완료 → {MODEL_PATH}")
